In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import xesmf as xe
from metpy.units import units
from metpy import calc as mpcalc
import netCDF4
from netCDF4 import Dataset
from datetime import datetime, date, timedelta

In [ ]:
real1_ds = xr.open_dataset('/langlab_rit/andrea/ERA5data/era5_202101_t.nc')
real2_ds = xr.open_dataset('/langlab_rit/andrea/ERA5data/era5_202102_t.nc')

In [ ]:
#Regridding ERA5 data using xesmf

ds_out = xr.Dataset(
    {
        "lon": (["lon"], np.arange(0, 360, 1.5)),
        "lat": (["lat"], np.arange(-90, 91.5, 1.5)),
    }
)

regridder = xe.Regridder(real1_ds, ds_out, "bilinear")
regridder

In [ ]:
real1_rgrd = regridder(real1_ds)
real2_rgrd = regridder(real2_ds)

In [ ]:
# Vertical level specificaton
vlevel = 1000
levelStr = str(vlevel)

#Box coordinates: [[49,-108], [49,-87], [32,-87], [32,-108],[49,-108]]
lat_min = 25
lat_max = 75
lon_min = 200
lon_max = 300

In [ ]:
#Reducing the datasets
r1 = real1_rgrd.reindex(lat=list(reversed(real1_rgrd.lat)))
t = r1['t'].sel(level=vlevel)#<--forecasted temps at 1000mb
rt1_rd = t.sel(time=t.time.dt.hour == 0)
rt1_rdf = rt1_rd.sel(lat=slice(lat_max,lat_min), lon=slice(lon_min,lon_max))

r2 = real2_rgrd.reindex(lat=list(reversed(real2_rgrd.lat)))
t2 = r2['t'].sel(level=vlevel)#<--forecasted temps at 1000mb
rt2_rd = t2.sel(time=t2.time.dt.hour == 0)
rt2_rdf = rt2_rd.sel(lat=slice(lat_max,lat_min), lon=slice(lon_min,lon_max))

In [ ]:
rt1_rdf

In [ ]:
rt2_rdf

In [ ]:
rt1_ds = rt1_rdf.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/box_proj/CONCAN_data/rd_t/realt_jan.nc",
                               mode='w')
rt2_ds = rt2_rdf.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/box_proj/CONCAN_data/rd_t/realt_feb.nc",
                               mode='w')

In [ ]:
rt1_ds = xr.open_dataset("/home11/grad/2021/gd673524/research/gdines/box_proj/CONCAN_data/rd_t/realt_jan.nc")
rt2_ds = xr.open_dataset("/home11/grad/2021/gd673524/research/gdines/box_proj/CONCAN_data/rd_t/realt_feb.nc")

In [ ]:
rt2_ds

In [ ]:
#Making lat/lon means
rt1_llmn = rt1_rdf.mean(dim=['lat','lon'])
rt2_llmn = rt2_rdf.mean(dim=['lat', 'lon'])

In [ ]:
rt1_llmn

In [ ]:
rt1_llmn = rt1_llmn.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/box_proj/CONCAN_data/llmn_t/realt_jan.nc",
                               mode='w')
rt2_llmn = rt2_llmn.to_netcdf(path="/home11/grad/2021/gd673524/research/gdines/box_proj/CONCAN_data/llmn_t/realt_feb.nc",
                               mode='w')